In [24]:
import pandas as pd
import numpy as np
import optuna
import dacon_law_class as dlc
from dacon_law_class import SimpleOps as so

import lightgbm as lgb
import re
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from xgboost.sklearn import XGBClassifier as xgb
# import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')


from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [3]:
# train
# test
sample_submission

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1235,TEST_1235,0
1236,TEST_1236,0
1237,TEST_1237,0
1238,TEST_1238,0


In [4]:
train.info()
print('\n\n\n')
# test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  2478 non-null   object
 1   first_party         2478 non-null   object
 2   second_party        2478 non-null   object
 3   facts               2478 non-null   object
 4   first_party_winner  2478 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 96.9+ KB






In [5]:
df = pd.DataFrame(train['facts'])
df = dlc.law_preprocessor(df, 'facts')
train['facts'] = df['facts']
df = pd.DataFrame(train['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
train['first_party'] = df['first_party']
df = pd.DataFrame(train['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
train['second_party'] = df['second_party']
train_cleansed = train.drop(columns='ID')
train_cleansed


,first_party,second_party,facts,first_party_winner
0,phil st amant,herman thompson,on june 27 1962 phil st amant cidate for publi...,1
1,stephen duncan,lawrence owens,ramon nelson was riding his bike when he suffe...,0
2,billy joe magwood,tony patterson warden,an alabama state court convicted billy joe mag...,1
3,linkletter,walker,victor linkletter was convicted in state court...,0
4,william earl fikes,alabama,on april 24 1953 in selma alabama an intruder ...,1
...,...,...,...,...
2473,hollyfrontier cheyenne refining llc,renewable fuels association,congress amended the clean air act through the...,1
2474,grupo mexicano de desarrollo,alliance bond fund,alliance bond fund an investment fund purchase...,1
2475,peguero,united states,in 1992 the district court sentenced manuel pe...,0
2476,immigration naturalization service,st cyr,on march 1996 enrico st cyr lawful permanent r...,0


In [6]:
df = pd.DataFrame(test['facts'])
df = dlc.law_preprocessor(df, 'facts')
test['facts'] = df['facts']
df = pd.DataFrame(test['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
test['first_party'] = df['first_party']
df = pd.DataFrame(test['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
test['second_party'] = df['second_party']
test_cleansed = test.drop(columns='ID')
test_cleansed

,first_party,second_party,facts
0,salerno,united states,the 1984 bail reform act allowed the federal c...
1,milberg weiss bershad hynes lerach,lexecon,lexecon was defendant in class action lawsuit ...
2,no 07 582 title federal communications commission,fox television stations,in 2002 2003 fox television stations broadcast...
3,harold kaufman,united states,during his trial for armed robbery of federall...
4,berger,hanlon,in 1993 magistrate judge issued warrant author...
...,...,...,...
1235,haitian centers council,chris sale acting commissioner immigration nat...,according to executive order no 12807 signed b...
1236,whitman,american trucking associations,section 109 of the clean air act requires the ...
1237,linda matteo john madigan,william barr,linda matteo john madigan created plan for uti...
1238,washington state apple advertising commission,hunt,in 1972 the north carolina board of agricultur...


## BERT

@article{turc2019,
  title={Well-Read Students Learn Better: On the Importance of Pre-training Compact Models},
  author={Turc, Iulia and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina},
  journal={arXiv preprint arXiv:1908.08962v2 },
  year={2019}
}


In [7]:
train_facts = pd.DataFrame(test_cleansed['facts'])
train_facts

,facts
0,the 1984 bail reform act allowed the federal c...
1,lexecon was defendant in class action lawsuit ...
2,in 2002 2003 fox television stations broadcast...
3,during his trial for armed robbery of federall...
4,in 1993 magistrate judge issued warrant author...
...,...
1235,according to executive order no 12807 signed b...
1236,section 109 of the clean air act requires the ...
1237,linda matteo john madigan created plan for uti...
1238,in 1972 the north carolina board of agricultur...


In [8]:
test_fact = pd.DataFrame(test_cleansed['facts'])
test_fact

,facts
0,the 1984 bail reform act allowed the federal c...
1,lexecon was defendant in class action lawsuit ...
2,in 2002 2003 fox television stations broadcast...
3,during his trial for armed robbery of federall...
4,in 1993 magistrate judge issued warrant author...
...,...
1235,according to executive order no 12807 signed b...
1236,section 109 of the clean air act requires the ...
1237,linda matteo john madigan created plan for uti...
1238,in 1972 the north carolina board of agricultur...


model_input_df = dlc.bert_tokenizer(train_facts, 'facts')

model_input_df.info()

df_part_1, df_part_2, df_part_3, df_part_4, df_part_5, df_part_6, df_part_7, df_part_8, df_part_9, df_part_10, df_part_11, df_part_12, df_part_13, df_part_14, df_part_15, df_part_16, df_part_17, df_part_18, df_part_19, df_part_20, df_part_21, df_part_22, df_part_23, df_part_24, df_part_25, df_part_26 = so.df_divider(train_cleansed, 'facts')
df_part_1 = pd.DataFrame(df_part_1)
df_part_2 = pd.DataFrame(df_part_2)
df_part_3 = pd.DataFrame(df_part_3)
df_part_4 = pd.DataFrame(df_part_4)
df_part_5 = pd.DataFrame(df_part_5)
df_part_6 = pd.DataFrame(df_part_6)
df_part_7 = pd.DataFrame(df_part_7)
df_part_8 = pd.DataFrame(df_part_8)
df_part_9 = pd.DataFrame(df_part_9)
df_part_10 = pd.DataFrame(df_part_10)
df_part_11 = pd.DataFrame(df_part_11)
df_part_12 = pd.DataFrame(df_part_12)
df_part_13 = pd.DataFrame(df_part_13)
df_part_14 = pd.DataFrame(df_part_14)
df_part_15 = pd.DataFrame(df_part_15)
df_part_16 = pd.DataFrame(df_part_16)
df_part_17 = pd.DataFrame(df_part_17)
df_part_18 = pd.DataFrame(df_part_18)
df_part_19 = pd.DataFrame(df_part_19)
df_part_20 = pd.DataFrame(df_part_20)
df_part_21 = pd.DataFrame(df_part_21)
df_part_22 = pd.DataFrame(df_part_22)
df_part_23 = pd.DataFrame(df_part_23)
df_part_24 = pd.DataFrame(df_part_24)
df_part_25 = pd.DataFrame(df_part_25)
df_part_26 = pd.DataFrame(df_part_26)
print(df_part_1)

embedded_df_1 = dlc.auto_tokenizer(train_cleansed, 'facts')
embedded_df_1

embedded_df_1 = embedded_df_1.rename(columns={0:'facts_berted'})

embedded_df_1.to_csv('./embeddings/facts_embedded.csv', index=False)

In [ ]:
first_party_berted = dlc.auto_tokenizer(train_cleansed, 'first_party')
first_party_berted = first_party_berted.rename(columns={0:'first_party_berted'})
first_party_berted.to_csv('./embeddings/first_party_berted.csv', index=False)

In [ ]:
second_party_berted = dlc.auto_tokenizer(train_cleansed, 'second_party')
second_party_berted = second_party_berted.rename(columns={0:'second_party_berted'})
second_party_berted.to_csv('./embeddings/second_party_berted.csv', index=False)

In [ ]:
facts_berted = dlc.auto_tokenizer(train_cleansed, 'facts')

In [ ]:
facts_berted = facts_berted.rename(columns={0:'facts_berted'})
facts_berted.to_csv('./embeddings/facts_berted.csv', index=False)

In [ ]:
first_party_berted = pd.read_csv('./embeddings/first_party_berted.csv')
second_party_berted = pd.read_csv('./embeddings/second_party_berted.csv')
facts_berted = pd.read_csv('./embeddings/facts_berted.csv')

In [ ]:
all_ready_to_ml = pd.concat([first_party_berted['first_party_berted'], second_party_berted['second_party_berted'], facts_berted['facts_berted'], train_cleansed['first_party_winner']], axis=1)
all_ready_to_ml.to_csv('./embeddings/1_train_ready_to_ml.csv', index=False)


In [ ]:
test_first_party_berted = dlc.auto_tokenizer(test_cleansed, 'first_party')
test_first_party_berted = test_first_party_berted.rename(columns={0:'first_party_berted'})
test_first_party_berted.to_csv('./embeddings/test_first_party_berted.csv', index=False)

In [ ]:
test_second_party_berted = dlc.auto_tokenizer(test_cleansed, 'second_party')
test_second_party_berted = test_second_party_berted.rename(columns={0:'second_party_berted'})
test_second_party_berted.to_csv('./embeddings/test_second_party_berted.csv', index=False)

In [ ]:
test_facts_berted = dlc.auto_tokenizer(test_cleansed, 'facts')
test_facts_berted = test_facts_berted.rename(columns={0:'facts_berted'})
test_facts_berted.to_csv('./embeddings/test_facts_berted.csv', index=False)

In [ ]:
test_first_party_berted = pd.read_csv('./embeddings/test_first_party_berted.csv')
test_second_party_berted = pd.read_csv('./embeddings/test_second_party_berted.csv')
test_facts_berted = pd.read_csv('./embeddings/test_facts_berted.csv')

In [ ]:
test_ready_to_ml = pd.concat([test_first_party_berted['first_party_berted'], test_second_party_berted['second_party_berted'], test_facts_berted['facts_berted']], axis=1)
test_ready_to_ml.to_csv('./embeddings/2_test_ready_to_ml.csv', index=False)

# 여기

In [10]:
train_to_ml = pd.read_csv('./embeddings/1_train_ready_to_ml.csv')
test_ready_to_ml = pd.read_csv('./embeddings/2_test_ready_to_ml.csv')

In [12]:
train_to_ml_fp_pr = dlc.tensor_separator(train_to_ml, 'first_party_berted')
train_to_ml_sp_pr = dlc.tensor_separator(train_to_ml, 'second_party_berted')
train_to_ml_facts_pr = dlc.tensor_separator(train_to_ml, 'facts_berted')

100%|███████████████████████████████████| 2478/2478 [00:00<00:00, 431336.54it/s]


In [13]:
train_to_ml_fp_pr = train_to_ml_fp_pr.astype('float64')
train_to_ml_sp_pr = train_to_ml_sp_pr.astype('float64') 
train_to_ml_facts_pr = train_to_ml_facts_pr.astype('float64')


In [14]:
train_to_ml_fp_pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2478 non-null   float64
 1   1       2478 non-null   float64
dtypes: float64(2)
memory usage: 38.8 KB


In [15]:
test_to_ml_fp_pr = dlc.tensor_separator(test_ready_to_ml, 'first_party_berted')
test_to_ml_sp_pr = dlc.tensor_separator(test_ready_to_ml, 'second_party_berted')
test_to_ml_facts_pr = dlc.tensor_separator(test_ready_to_ml, 'facts_berted')

100%|███████████████████████████████████| 1240/1240 [00:00<00:00, 248207.36it/s]


In [16]:
test_to_ml_fp_pr = test_to_ml_fp_pr.astype('float64')
test_to_ml_sp_pr = test_to_ml_sp_pr.astype('float64')
test_to_ml_facts_pr = test_to_ml_facts_pr.astype('float64')

In [20]:
to_be_X = pd.concat([train_to_ml_fp_pr, train_to_ml_sp_pr, train_to_ml_facts_pr], axis=1)
to_be_test_x = pd.concat([test_to_ml_fp_pr, test_to_ml_sp_pr, test_to_ml_facts_pr], axis=1)


In [21]:
to_be_X.columns = np.arange(len(to_be_X.columns))
to_be_X = pd.concat([to_be_X, train_to_ml['first_party_winner']], axis =1)
to_be_X

,0,1,2,3,4,5,first_party_winner
0,-0.7554,-0.6552,-0.9958,-0.0918,-0.1506,-0.9886,1
1,-0.2418,-0.9703,-0.9404,-0.3401,-0.4364,-0.8998,0
2,0.3450,-0.9386,-0.9466,-0.3223,-0.2305,-0.9731,1
3,-0.8383,-0.5452,0.1442,-0.9896,0.0159,-0.9999,0
4,-0.7003,-0.7138,-0.9851,0.1720,-0.7125,-0.7016,1
...,...,...,...,...,...,...,...
2473,-0.8816,-0.4721,-0.7886,0.6150,-0.6131,-0.7900,1
2474,0.8924,-0.4513,-0.9831,0.1829,-0.4205,-0.9073,1
2475,0.9199,0.3922,-0.9851,0.1720,-0.1907,-0.9816,0
2476,-0.9989,0.0467,-0.9452,-0.3264,-0.4114,-0.9114,0


In [22]:
to_be_test_x.columns = np.arange(len(to_be_test_x.columns))
to_be_test_x

,0,1,2,3,4,5
0,0.9199,0.3922,0.9992,0.0394,0.8901,-0.4557
1,0.6421,0.7667,0.8362,-0.5484,0.1711,-0.9853
2,0.9467,0.3221,0.9988,-0.0484,0.9713,-0.2379
3,0.9946,0.1042,0.9992,0.0394,0.3317,-0.9434
4,0.9927,-0.1205,0.7966,-0.6045,0.1093,-0.9940
...,...,...,...,...,...,...
1235,0.6386,0.7696,0.9959,-0.0905,0.6906,-0.7232
1236,0.8192,-0.5736,0.9941,0.1083,0.5300,-0.8480
1237,0.8967,0.4426,0.9083,0.4183,0.5030,-0.8643
1238,0.6885,0.7252,0.9724,-0.2334,0.5845,-0.8114


In [25]:
!mkdir -p 'model'
predictor = TabularPredictor(label='first_party_winner',path='model').fit(to_be_X, presets=['best_quality'])


Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "model/"
AutoGluon Version:  0.8.0
Python Version:     3.10.9
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.5.0: Mon Apr 24 20:53:44 PDT 2023; root:xnu-8796.121.2~5/RELEASE_ARM64_T8103
Disk Space Avail:   38.91 GB / 245.11 GB (15.9%)
Train Data Rows:    2478
Train Data Columns: 6
Label Column: first_party_winner
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators 

In [26]:
preds_solid = predictor.predict(to_be_test_x)
preds_proba = predictor.predict_proba(to_be_test_x)


In [27]:
print(preds_solid.value_counts())
# preds_proba


1    992
0    248
Name: first_party_winner, dtype: int64


In [28]:
sample_submission

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1235,TEST_1235,0
1236,TEST_1236,0
1237,TEST_1237,0
1238,TEST_1238,0


In [29]:
final_submission = pd.concat([sample_submission['ID'], preds_solid], axis=1)
final_submission

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,1
2,TEST_0002,0
3,TEST_0003,1
4,TEST_0004,1
...,...,...
1235,TEST_1235,1
1236,TEST_1236,1
1237,TEST_1237,1
1238,TEST_1238,1


In [30]:
final_submission.to_csv("./bert_autogluon_submission.csv", index=False)